In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
html_text = requests.get('https://www.emploitic.com/offres-d-emploi', headers=headers).text
# print(html_text)


In [28]:
soup = BeautifulSoup(html_text, 'html.parser')
jobs = soup.find_all('li', class_ ='separator-bot')
jobs

[<li class="separator-bot">
 <!--<ul class="share-list hidden-phone">
 		<li class="share-featured" onClick="" data-toggle="modal" data-target="#loginModal"><i class="fa fa-star"></i><span class="text">Sauvegarder</span></li>
                 <li class="share-mail"><span class="text"><a class="color-a" href="mailto:?subject=Annonce consultée sur Emploitic.com - Coordinateur Trésorerie et Relations Bancaires&body=Bonjour,%0D%0A%0D%0AVoici une annonce d’Emploitic.com qui me semble intéressante : Coordinateur Trésorerie et Relations Bancaires : https://www.emploitic.com/offres-d-emploi/offre-d-emploi/algerie/alger/bab-el-oued/10076659-coordinateur-tresorerie-et-relations-bancaires">Envoyer par mail</a></span><i class="fa fa-envelope-o"></i></li>
         <li class="share-sn">
             <i class="fa fa-mail-forward"></i>
             <span class="text">
                 <a href="http://www.facebook.com/sharer/sharer.php?s=100&p[url]=https%3A%2F%2Fwww.emploitic.com%2F%2Foffres-d-emploi%2

In [44]:
soup = BeautifulSoup(html_text, 'html.parser')
job = soup.find('li', class_ ='separator-bot')




In [56]:
job_name = job.find('h2', class_ = 'ellipsis row-fluid').text.replace('\n ','').replace(' ','')
job_name

'CoordinateurTrésorerieetRelationsBancaires'

In [36]:
company_name = job.find('h6', class_ = 'ellipsis').text
company_name

'Entreprise anonyme'

In [39]:
work_shift_experience = job.find('span', class_ ='spaced-right phone-display-blok').text
work_shift_experience

" Responsable d'équipe (Plus de 10 ans)"

In [59]:
localisation = job.find('span', class_ ='spaced-right').text
localisation

'Bab El Oued, Alger, Algérie...'

In [61]:
publish_date= job.find('span', class_ ='spaced-right pull-left').text
publish_date

" Aujourd'hui"

In [62]:
print(f'''
company name: {company_name}
job name: {job_name}
work shift and experience: {work_shift_experience}
localisation: {localisation}
publish date: {publish_date}
      ''')


company name: Entreprise anonyme
job name: CoordinateurTrésorerieetRelationsBancaires
work shift and experience:  Responsable d'équipe (Plus de 10 ans)
localisation: Bab El Oued, Alger, Algérie...
publish date:  Aujourd'hui
      


take all jobs in first page

In [98]:
# def find_jobs():  
soup = BeautifulSoup(html_text, 'html.parser')
jobs = soup.find_all('li', class_ ='separator-bot')
for job in jobs:
            job_name = job.find('h2', class_ = 'ellipsis row-fluid').text.replace('\n          ','')
            company_name = job.find('h6', class_ = 'ellipsis').text
            work_shift_experience = job.find('span', class_ ='spaced-right phone-display-blok').text
            localisation = job.find('span', class_ ='spaced-right').text
            publish_date= job.find('span', class_ ='spaced-right pull-left').text
            more_info_link = job.div.a['href']
            

            print(f"company name: {company_name}")
            print(f"job name: {job_name}")
            print(f"work shift and experience: {work_shift_experience}")
            print(f"localisation: {localisation}")
            print(f"publish date: {publish_date}")
            print(f"more_info: {more_info_link}")
        # Access the linked page and extract additional information
            linked_page_content = requests.get(more_info_link, headers=headers).text
            linked_page_soup = BeautifulSoup(linked_page_content, 'html.parser')
            
            # Extract additional information from the linked page
            # additional_info = linked_page_soup.find('div', class_='content separator-top').text.strip()
            # print(f"additional information: {additional_info}")
            additional_info_element = linked_page_soup.find('section', class_='content styled spaced details-description span8')

            if additional_info_element:
                additional_info = additional_info_element.text.strip()
                print(f"additional information: {additional_info}")
            else:
                print("No additional information found on this page.")

            print(' ')

# if __name__ == '__main__':
#     while True:
#         find_jobs()  
#         time_wait = 10    
#         print(f'Waiting {time_wait} minutes....')
#         time.sleep(time_wait * 60)

company name: Altius Contact
job name:               Superviseur dans le domaine de la télévente                    
work shift and experience:  Responsable d'équipe (6 à 10 ans)
localisation: Tizi Ouzou, Algérie
publish date:  Aujourd'hui
more_info: https://www.emploitic.com/offres-d-emploi/offre-d-emploi/algerie/tizi-ouzou/10076662-superviseur-dans-le-domaine-de-la-televente
No additional information found on this page.
 
company name: Entreprise anonyme
job name:               Coordinateur Trésorerie et Relations Bancaires                    
work shift and experience:  Responsable d'équipe (Plus de 10 ans)
localisation: Bab El Oued, Alger, Algérie...
publish date:  Aujourd'hui
more_info: https://www.emploitic.com/offres-d-emploi/offre-d-emploi/algerie/alger/bab-el-oued/10076659-coordinateur-tresorerie-et-relations-bancaires
additional information: Missions Principales :Veiller à la solvabilité de l’entreprise.Assurer la liquidité financière.  Gérer la relation bancaire et représent

avec pagination

In [4]:
import csv

# Open a CSV file in write mode
with open('jobs_data.csv', 'w', newline='', encoding='utf-8') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)

    # Write the header row
    csv_writer.writerow(['Company Name', 'Job Name', 'Work Shift and Experience', 'Localisation', 'Publish Date', 'More Info', 'Additional Information'])

    page_number = 0
    while True:
        # Construct the URL for the current page
        url = f'https://www.emploitic.com/offres-d-emploi?start={page_number * 20}'
        response = requests.get(url, headers=headers)
        
        # Check if the page request was successful
        if response.status_code != 200:
            print(f"Failed to retrieve page {page_number}. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        
        jobs = soup.find_all('li', class_='separator-bot')

        if not jobs:
            # Break out of the loop if there are no more jobs
            break

        for job in jobs:
            # Extract job information (add error handling if needed)
            try:
                job_name = job.find('h2', class_='ellipsis row-fluid').text.replace('\n          ', '')
                company_name = job.find('h6', class_='ellipsis').text
                work_shift_experience = job.find('span', class_='spaced-right phone-display-blok').text
                localisation = job.find('span', class_='spaced-right').text
                publish_date = job.find('span', class_='spaced-right pull-left').text
                more_info_link = job.div.a['href']
                
                # Access the linked page and extract additional information
                linked_page_content = requests.get(more_info_link, headers=headers).text
                linked_page_soup = BeautifulSoup(linked_page_content, 'html.parser')

                additional_info_element_1 = linked_page_soup.find('section', class_='content styled spaced details-description span8')
                additional_info_element_2 = linked_page_soup.find('div', class_='content separator-top')

                if additional_info_element_1:
                    additional_info = additional_info_element_1.text.strip()
                elif additional_info_element_2:
                    additional_info = additional_info_element_2.text.strip()
                else:
                    additional_info = None

                # Write the data to the CSV file
                csv_writer.writerow([company_name, job_name, work_shift_experience, localisation, publish_date, more_info_link, additional_info])
            except Exception as e:
                print(f"Error processing job on page {page_number}: {e}")

        # Move to the next page
        page_number += 1

# Print a message indicating that the data has been saved to the CSV file
print("Data has been saved to jobs_data.csv")


KeyboardInterrupt: 